![image](https://raw.githubusercontent.com/IBM/watson-machine-learning-samples/master/cloud/notebooks/headers/watsonx-Prompt_Lab-Notebook.png)
# Agents Lab Notebook v1.0.0
This notebook contains steps and code to demonstrate the use of agents
configured in Agent Lab in watsonx.ai. It introduces Python API commands
for authentication using API key and invoking a LangGraph agent with a watsonx chat model.

**Note:** Notebook code generated using Agent Lab will execute successfully.
If code is modified or reordered, there is no guarantee it will successfully execute.
For details, see: <a href="/docs/content/wsj/analyze-data/fm-prompt-save.html?context=wx" target="_blank">Saving your work in Agent Lab as a notebook.</a>

Some familiarity with Python is helpful. This notebook uses Python 3.11.

## Notebook goals
The learning goals of this notebook are:

* Defining a Python function for obtaining credentials from the IBM Cloud personal API key
* Creating an agent with a set of tools using a specified model and parameters
* Invoking the agent to generate a response 

# Setup

In [ ]:
# import dependencies
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai import APIClient
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from ibm_watsonx_ai.foundation_models.utils import Tool, Toolkit
import json
import requests

## watsonx API connection
This cell defines the credentials required to work with watsonx API for Foundation
Model inferencing.

**Action:** Provide the IBM Cloud personal API key. For details, see
<a href="https://cloud.ibm.com/docs/account?topic=account-userapikey&interface=ui" target="_blank">documentation</a>.


In [ ]:
import os
import getpass

def get_credentials():
	return {
		"url" : "https://au-syd.ml.cloud.ibm.com",
		"apikey" : getpass.getpass("Please enter your api key (hit enter): ")
	}

def get_bearer_token():
    url = "https://iam.cloud.ibm.com/identity/token"
    headers = {"Content-Type": "application/x-www-form-urlencoded"}
    data = f"grant_type=urn:ibm:params:oauth:grant-type:apikey&apikey={credentials['apikey']}"

    response = requests.post(url, headers=headers, data=data)
    return response.json().get("access_token")

credentials = get_credentials()

# Using the agent
These cells demonstrate how to create and invoke the agent
with the selected models, tools, and parameters.

## Defining the model id
We need to specify model id that will be used for inferencing:

In [ ]:
model_id = "meta-llama/llama-3-2-11b-vision-instruct"

## Defining the model parameters
We need to provide a set of model parameters that will influence the
result:

In [ ]:
parameters = {
    "frequency_penalty": 0,
    "max_tokens": 2000,
    "presence_penalty": 0,
    "temperature": 0,
    "top_p": 1
}

## Defining the project id or space id
The API requires project id or space id that provides the context for the call. We will obtain
the id from the project or space in which this notebook runs:

In [ ]:
project_id = os.getenv("PROJECT_ID")
space_id = os.getenv("SPACE_ID")


## Creating the agent
We need to create the agent using the properties we defined so far:

In [ ]:
client = APIClient(credentials=credentials, project_id=project_id, space_id=space_id)

# Create the chat model
def create_chat_model():
    chat_model = ChatWatsonx(
        model_id=model_id,
        url=credentials["url"],
        space_id=space_id,
        project_id=project_id,
        params=parameters,
        watsonx_client=client,
    )
    return chat_model

In [ ]:
from ibm_watsonx_ai.deployments import RuntimeContext

context = RuntimeContext(api_client=client)


vector_index_id = "60f79086-f9fa-4049-bbe2-97a8e50d4294"

def create_rag_tool(vector_index_id, api_client):
    config = {
        "vectorIndexId": vector_index_id,
        "projectId": project_id
    }

    tool_description = "Search information in documents to provide context to a user query. Useful when asked to ground the answer in specific knowledge about 61892e12-3a4b-46cc-9e34-c2ddbd1806e3_intents"
    
    return create_utility_agent_tool("RAGQuery", config, api_client, tool_description=tool_description)



def create_utility_agent_tool(tool_name, params, api_client, **kwargs):
    from langchain_core.tools import StructuredTool
    utility_agent_tool = Toolkit(
        api_client=api_client
    ).get_tool(tool_name)

    tool_description = utility_agent_tool.get("description")

    if (kwargs.get("tool_description")):
        tool_description = kwargs.get("tool_description")
    elif (utility_agent_tool.get("agent_description")):
        tool_description = utility_agent_tool.get("agent_description")
    
    tool_schema = utility_agent_tool.get("input_schema")
    if (tool_schema == None):
        tool_schema = {
            "type": "object",
            "additionalProperties": False,
            "$schema": "http://json-schema.org/draft-07/schema#",
            "properties": {
                "input": {
                    "description": "input for the tool",
                    "type": "string"
                }
            }
        }
    
    def run_tool(**tool_input):
        query = tool_input
        if (utility_agent_tool.get("input_schema") == None):
            query = tool_input.get("input")

        results = utility_agent_tool.run(
            input=query,
            config=params
        )
        
        return results.get("output")
    
    return StructuredTool(
        name=tool_name,
        description = tool_description,
        func=run_tool,
        args_schema=tool_schema
    )


def create_custom_tool(tool_name, tool_description, tool_code, tool_schema, tool_params):
    from langchain_core.tools import StructuredTool
    import ast

    def call_tool(**kwargs):
        tree = ast.parse(tool_code, mode="exec")
        custom_tool_functions = [ x for x in tree.body if isinstance(x, ast.FunctionDef) ]
        function_name = custom_tool_functions[0].name
        compiled_code = compile(tree, 'custom_tool', 'exec')
        namespace = tool_params if tool_params else {}
        exec(compiled_code, namespace)
        return namespace[function_name](**kwargs)
        
    tool = StructuredTool(
        name=tool_name,
        description = tool_description,
        func=call_tool,
        args_schema=tool_schema
    )
    return tool

def create_custom_tools():
    custom_tools = []


def create_tools(context):
    tools = []
    tools.append(create_rag_tool(vector_index_id, client))
    
    config = None
    tools.append(create_utility_agent_tool("GoogleSearch", config, client))
    config = {
    }
    tools.append(create_utility_agent_tool("DuckDuckGo", config, client))
    config = {
        "maxResults": 5
    }
    tools.append(create_utility_agent_tool("Wikipedia", config, client))

    return tools

In [ ]:
def create_agent(context):
    # Initialize the agent
    chat_model = create_chat_model()
    tools = create_tools(context)

    memory = MemorySaver()
    instructions = """# General Behavior Guidelines

- You are an AI assistant that provides accurate, safe, and user-friendly information.
- Follow these rules for all interactions unless overridden by agent-specific instructions.

## 1. Greeting & Conversation Etiquette
- Always greet warmly at the start of the conversation.
- Maintain a friendly, helpful, and respectful tone.
- Encourage the user to share more details for better recommendations.
- If the user returns after a previous conversation, acknowledge them and, if possible, remember their preferences.

## 2. Tool Usage
- Use tools like Watsonx.ai Prompt Lab, JSON lookups, or integrated APIs for generating responses.
- Use markdown syntax for formatting code snippets, links, JSON, tables, images, or files.
- Any HTML tags must be wrapped in block quotes, for example:
  ```<html>```
- When returning code blocks, specify the programming language.
- For calculations or structured lookups, always call the appropriate tool instead of manually computing.
- If a tool fails, try a different tool or adjust the query before declaring the problem unsolvable.

## 3. Image Handling
- If a tool returns an IMAGE in the result, include it in your answer as Markdown.
- Example:
  Tool result: IMAGE({commonApiUrl}/wx/v1-beta/utility_agent_tools/cache/images/example.png)
  Markdown to return:
  ![Generated image]({commonApiUrl}/wx/v1-beta/utility_agent_tools/cache/images/example.png)

## 4. Accuracy & Safety
- Ensure all nutrition-related information is factually correct.
- Never recommend foods containing allergens that the user has specified.
- Avoid medical claims and suggest professional consultation for specific health conditions.
- Only provide dietary recommendations that are safe for general audiences.

## 5. Clarity & Structure
- Use simple, understandable language suitable for all audiences.
- Present meal plans and advice using bullet points or numbered lists.
- Avoid long paragraphs; keep responses structured for easy reading.

## 6. Personalization & Continuity
- Always adapt responses based on the user's goals, dietary preferences, and allergies.
- Remember previously provided preferences within the same conversation.
- Offer alternatives if a suggested food item is not suitable for the user.

## 7. Ethics & Respect
- Be inclusive and culturally sensitive.
- Avoid stereotypes and biased assumptions in meal planning.
- Show empathy and encouragement in all responses.

## 8. Engagement & Feedback
- After giving a meal plan, ask if the user would like more options or adjustments.
- Invite them to provide feedback for better personalization.
- Keep the interaction two-way, not just one-sided advice.

You are Nutrition_Assistant, a smart, empathetic, and interactive AI-powered virtual nutritionist built using Watsonx.ai and IBM Watson Assistant. You specialize in generating personalized meal plans and providing nutritional guidance.

Your main role is to interact with users conversationally, collect relevant health and lifestyle information, and generate customized recommendations. You should:

1. Greeting & Conversation Start:
   - When greeted, say: 
\"Hi, I am Watsonx.ai Nutrition Assistant. I can help you create a meal plan tailored to your needs. First, what is your primary health goal? For example, weight loss, muscle gain, or a balanced diet.\"
   - If the user asks for help with nutrition or a meal plan, first ask:
     \"Great! What is your primary health goal? For example, weight loss, muscle gain, or a balanced diet.\"
   - After goal, ask:
     \"Got it. Do you have any dietary preferences, like vegetarian, vegan, or non-vegetarian?\"
- After they answer, ask:
     \"Thanks! Do you have any allergies I should be aware of?\"
   - Only after collecting these details, move on to generating the meal plan.


2. Information Gathering:
   - Collect the following details from the user:
     - Health goal (e.g., weight loss, muscle gain, balanced diet, managing a medical condition)
     - Dietary preference (e.g., vegetarian, vegan, non-vegetarian, spicy, sweet)
     - Allergies (e.g., gluten, dairy, nuts, soy)
     - Lifestyle/activity level (e.g., sedentary, moderately active, highly active)
     - Optional: preferred cuisine or cultural food preferences

3. Personalization Rules:
   - Always avoid suggesting foods containing allergens provided by the user.
   - Adjust meals to align with both dietary preferences and cultural background.
   - Ensure recommendations are realistic and accessible for most households.

4. Meal Plan Generation:
   - Use Watsonx.ai Prompt Lab to create dynamic meal plans when user data is complete.
   - Each meal plan should include:
       - Breakfast
       - Lunch
       - Dinner
       - Optional snacks
       -suggestions(like exercises and dietary) (if relevant)
   - Provide short explanations for why each meal is chosen, focusing on health benefits and relevance to the user's goal.
   - Suggest alternative food swaps when possible.

5. Multimodal Capability:
   - Be prepared to interpret and use information from text, voice, and optionally food-related images (e.g., grocery labels, meal photos).

6. Feedback & Adaptation:
   - After presenting a meal plan, ask: \"Would you like me to adjust this plan or provide alternatives?\"
   - If feedback is given, adjust recommendations accordingly.
   - Maintain a friendly, encouraging tone throughout.

7. Style & Tone:
   - Be professional, empathetic, supportive, and motivating.
   - Use simple, clear language without heavy technical jargon.
   - Use emojis sparingly to keep the tone warm but not unprofessional.

8. Safety:
   - Avoid giving medical diagnoses or unsafe diet advice.
   - For serious or specific health conditions, politely recommend consulting a certified healthcare professional.
"""

    agent = create_react_agent(chat_model, tools=tools, checkpointer=memory, state_modifier=instructions)

    return agent

In [ ]:
# Visualize the graph
from IPython.display import Image, display
from langchain_core.runnables.graph import CurveStyle, MermaidDrawMethod, NodeStyles

Image(
    create_agent(context).get_graph().draw_mermaid_png(
        draw_method=MermaidDrawMethod.API,
    )
)


## Invoking the agent
Let us now use the created agent, pair it with the input, and generate the response to your question:


In [ ]:
agent = create_agent(context)

def convert_messages(messages):
    converted_messages = []
    for message in messages:
        if (message["role"] == "user"):
            converted_messages.append(HumanMessage(content=message["content"]))
        elif (message["role"] == "assistant"):
            converted_messages.append(AIMessage(content=message["content"]))
    return converted_messages

question = input("Question: ")

messages = [{
    "role": "user",
    "content": question
}]

generated_response = agent.invoke(
    { "messages": convert_messages(messages) },
    { "configurable": { "thread_id": "42" } }
)

print_full_response = False

if (print_full_response):
    print(generated_response)
else:
    result = generated_response["messages"][-1].content
    print(f"Agent: {result}")


# Next steps
You successfully completed this notebook! You learned how to use
watsonx.ai inferencing SDK to generate response from the foundation model
based on the provided input, model id and model parameters. Check out the
official watsonx.ai site for more samples, tutorials, documentation, how-tos, and blog posts.

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2024 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs (or equivalent) and License Information document for watsonx.ai Auto-generated Notebook (License Terms), such agreements located in the link below. Specifically, the Source Components and Sample Materials clause included in the License Information document for watsonx.ai Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="https://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BYC7LF" target="_blank">License Terms</a>  